# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [12]:
cities_df = pd.read_csv("../output_data/cities.csv")
cities_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Tongren,27.7172,109.1853,89.87,63,55,7.43,CN,1658377437
1,1,Bengkulu,-3.8004,102.2655,83.68,68,24,3.98,ID,1658377437
2,2,Tommot,58.9564,126.2925,78.40,55,13,4.36,RU,1658377438
3,3,Cherskiy,68.7500,161.3000,47.57,40,14,19.39,RU,1658377438
4,4,Yellowknife,62.4560,-114.3525,66.72,42,20,10.36,CA,1658377438
...,...,...,...,...,...,...,...,...,...,...
561,561,Arari,-3.4536,-44.7800,70.41,82,32,2.71,BR,1658377651
562,562,Villa Unión,23.2000,-106.2333,86.92,79,40,5.75,MX,1658377652
563,563,Humaitá,-7.5061,-63.0208,73.04,56,6,0.92,BR,1658377652
564,564,Banjar,-8.1900,114.9675,86.70,56,74,3.11,ID,1658377652


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
gmaps.configure(api_key=g_key)

In [14]:
locations = cities_df[["Lat", "Lng"]].astype(float)
Humidity = cities_df['Humidity'].astype(float)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#weather criteria: 1) max temperature lower than 80 degrees but higher than 70. 
                 # 2) Wind speed less than 10 mph.
                 # 3) lower then 5% cloudiness
hotel_df=cities_df.loc[(cities_df['Max Temp']<80) & (cities_df['Max Temp']>70) & (cities_df['Wind Speed']<5) 
                   & (cities_df['Cloudiness']<5)]
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
41,41,Montefiascone,42.5393,12.0362,70.34,91,0,2.06,IT,1658377449
50,50,Cayenne,4.9333,-52.3333,78.10,1,0,1.14,GF,1658377452
132,132,Bethel,41.3712,-73.4140,79.63,81,1,3.58,US,1658377477
277,277,Ukiah,39.1502,-123.2078,71.37,66,0,4.70,US,1658377523
375,375,Revelstoke,50.9832,-118.2023,71.62,64,0,3.44,CA,1658377564
443,443,Pachino,36.7186,15.0907,78.06,69,0,4.09,IT,1658377601
459,459,Palmer,42.1584,-72.3287,79.97,77,0,4.88,US,1658377614
486,486,Erzin,36.9559,36.2000,77.74,75,0,3.49,TR,1658377628
488,488,Powell River,49.8496,-124.5361,73.67,73,0,2.35,CA,1658377629


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df['Hotel Name']=""
hotel_df

/var/folders/r3/ljt9syw176jcr74xd342l8r40000gn/T/ipykernel_29261/2560813339.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name']=""


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
41,41,Montefiascone,42.5393,12.0362,70.34,91,0,2.06,IT,1658377449,
50,50,Cayenne,4.9333,-52.3333,78.10,1,0,1.14,GF,1658377452,
132,132,Bethel,41.3712,-73.4140,79.63,81,1,3.58,US,1658377477,
277,277,Ukiah,39.1502,-123.2078,71.37,66,0,4.70,US,1658377523,
375,375,Revelstoke,50.9832,-118.2023,71.62,64,0,3.44,CA,1658377564,
443,443,Pachino,36.7186,15.0907,78.06,69,0,4.09,IT,1658377601,
459,459,Palmer,42.1584,-72.3287,79.97,77,0,4.88,US,1658377614,
486,486,Erzin,36.9559,36.2000,77.74,75,0,3.49,TR,1658377628,
488,488,Powell River,49.8496,-124.5361,73.67,73,0,2.35,CA,1658377629,


In [7]:

# dictionary of parameters
parameters = {
    "type": "lodging",
    "keyword":"hotel",
    "radius": 5000,
    "key": g_key
}


# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    parameters["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotelresponse = requests.get(base_url, params=parameters).json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotelresponse["results"][0]["name"]
        print(f"{hotelresponse['results'][0]['name']}")
    
    except (KeyError, IndexError):
        print("Missing hotel info... skipping.")

Il Caminetto Resort
Grand Hotel Montabo
Hampton Inn Danbury
Hampton Inn Ukiah
Sutton Place Hotel Revelstoke Mountain Resort
Hotel Vittorio
The Trainmaster's Inn
Hattusa Vacation Thermal Club Erzin
Powell River Town Centre Hotel


In [8]:
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
41,41,Montefiascone,42.5393,12.0362,70.34,91,0,2.06,IT,1658377449,Il Caminetto Resort
50,50,Cayenne,4.9333,-52.3333,78.10,1,0,1.14,GF,1658377452,Grand Hotel Montabo
132,132,Bethel,41.3712,-73.4140,79.63,81,1,3.58,US,1658377477,Hampton Inn Danbury
277,277,Ukiah,39.1502,-123.2078,71.37,66,0,4.70,US,1658377523,Hampton Inn Ukiah
375,375,Revelstoke,50.9832,-118.2023,71.62,64,0,3.44,CA,1658377564,Sutton Place Hotel Revelstoke Mountain Resort
443,443,Pachino,36.7186,15.0907,78.06,69,0,4.09,IT,1658377601,Hotel Vittorio
459,459,Palmer,42.1584,-72.3287,79.97,77,0,4.88,US,1658377614,The Trainmaster's Inn
486,486,Erzin,36.9559,36.2000,77.74,75,0,3.49,TR,1658377628,Hattusa Vacation Thermal Club Erzin
488,488,Powell River,49.8496,-124.5361,73.67,73,0,2.35,CA,1658377629,Powell River Town Centre Hotel


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))